# PREPROCESSING BIM DATA TO LINKED DATA
In this notebook, we evaluate the BIM assets in a session repository.
For every ifcElement in an ifcfile, a BIMNODE metadata class is created 
that governs the metadata of the BIM element (paths, pose, etc.).
As ouput, the method generates RDF GRAPHS (.ttl) and O3D.MESH (.obj) files

>This codebase operates on the scan2bim2.yml environment (python 3.8) and ifcopenshell

In [1]:
#IMPORT PACKAGES
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

import os.path, time
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
import geomapi.tools as tl
from geomapi.utils import geometryutils as gmu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## 1. INITIALIZE SESSION

In [4]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"test" )#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = os.path.join(projectPath,"testfiles") #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
graphPath = os.path.join(sessionPath,"bimGraph1.ttl")
resourcePath=os.path.join(sessionPath,"resources")

phase='BIM-UF'
accuracy=0.05
# classes= '.IfcBeam | .IfcColumn | .IfcWall | .IfcSlab'#'.IfcElement' #

## 2. READ IFC FILES 
A BIMNode is created per ifcElement in the session. Mesh geometry is created for every node

In [5]:
ifcPath1=os.path.join(sessionPath,'IFC','Academiestraat_building_1.ifc')
ifcPath2=os.path.join(sessionPath,'IFC',"Mariakerke_AWV_Conform_3D_BT_l72.ifc")  
ifcPath3=os.path.join(sessionPath,'IFC',"Academiestraat_parking.ifc")  
ifcPath4=os.path.join(sessionPath,'IFC',"B1_ALG_Model.ifc")  

graphPath1 = os.path.join(sessionPath,"bimGraph1.ttl")
graphPath2 = os.path.join(sessionPath,"bimGraph2.ttl")
graphPath3 = os.path.join(sessionPath,"bimGraph3.ttl")
graphPath4 = os.path.join(sessionPath,"bimGraph4.ttl")

bimpath=os.path.join(sessionPath,"BIM")



In [6]:
#ifc3
ifc=ifcopenshell.open(ifcPath2)   
a=0
selector = Selector()
nodelist=[]
for ifcElement in selector.parse(ifc, '.ifcObject'):
    node=BIMNode(resource=ifcElement, ifcPath=ifcPath2,phase=phase,accuracy=accuracy, lod=300)
    node.save_resource(os.path.join(sessionPath,"BIMtest"))
    nodelist.append(node)
    print(node.subject)
    a+=1
    if a==100:
        break
   

file:///BT5_Rioolpijp_Aansluiting_WRP2_0zVea_7dP248qp_1F_Pwch
file:///BT1_Onderbouw_Fundering_2vLNF4ypr1AxRTiBL6oGaV
file:///BT5_Rioolpijp_Aansluiting_WRP2_14e880Gfj5GPb8b3TpG1KT
file:///BT1_Onderbouw_Fundering_3Cgs0xWeXE_eicfza_5qQT
file:///BT9_Deksel_Prive_Huisaansluiting_WPI22_2dpn5DCfn1dQru1BA3q36r
file:///BT5_Rioolpijp_Aansluiting_WRP2_2F6EQDaKz39ARyiADh_9v_
file:///BT8_Loofboom_Laag_WGI2_2aQMlZejr2ZPwOlbJD8cvR
file:///BT5_Rioolpijp_Aansluiting_WRP2_1cClrsMXT6J8Jsvtaoqt2a
file:///BT1_Onderbouw_Fundering_1n_3RrgP59tB3ZdWRm77Wq
file:///BT9_Deksel_Prive_Huisaansluiting_WPI22_0JMve7Yh595gmEQlxhR7Eb
file:///BT1_Onderbouw_Fundering_0jLihvjbj3jO6QBgJ8M5et
file:///BT9_Deksel_Prive_Huisaansluiting_WPI22_2WUk6SOuP77h2rmEuAXpk4
file:///BT8_Loofboom_Laag_WGI2_2gFVkBjcL9XARvz30P5ZPw
file:///BT9_Deksel_Prive_Huisaansluiting_WPI22_2fWnjw29HCdATDk4mDNxDe
file:///BT8_Loofboom_Laag_WGI2_2J_7gS4cHC08lLLAG8Cp_q
file:///BT5_Rioolpijp_Aansluiting_WRP2_1KuzWboA9BIBSW8R10MlKs
file:///BT5_Rioolpijp_Aanslu

In [7]:
geometries=[node.resource for node in nodelist if node.resource]

In [8]:
o3d.visualization.draw_geometries(geometries)

In [14]:
# Retrieve all (.ifc) files in the session
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
nodelist=[]

for path in allSessionFilePaths:
    if path.endswith(".ifc"): 
        list=tl.ifc_to_nodes(ifcPath=path,classes='ifcSite',getResource=True, phase=phase,accuracy=accuracy)
        nodelist.extend(list)
        for node in list:
            node.sessionPath=sessionPath
            node.awvLayer = node.name
            node.name = node.name  + "_" + node.globalId
            
print(str(len(nodelist))+ ' BIMNodes are created')
print('Depending on getGeometry setting, these nodes do not yet contain data but only the metadata')

IfcOpenShell parsing error. Note that an error is generated when no objects can be parsed from the selector. E.g. parsing for .ifcWall elements without the presence of ifcWalls will generate an error 


In [31]:
for node in nodelist:
    node.accuracy=accuracy
    node.phase=phase
    if 'l72'in node.ifcPath:
        node.coordinateSystem='Lambert72'
    elif 'l2008' in node.ifcPath:
        node.coordinateSystem='Lambert2008'
    elif 'wgs84' in node.ifcPathh:
        node.coordinateSystem='geospatial-wgs84'
    else:
        node.coordinateSystem='local'
print('Node coordinate systems have been set based on file names (l72, l2008, wgs84, local)!')

Node coordinate systems have been set based on file names (l72, l2008, wgs84, local)!


## 3. Create and write MESH FILES 
create mesh geometry if not yet available and write it in the location of the sessionPath

In [32]:
#Read e57 files, create o3d point clouds and write them to \\PCD\\'pcdnode.name'.pcd
newCounter=0
oldCounter=0
for node in nodelist:
    name=node.name.replace(':','_')
    path=sessionPath+'\\BIM\\'+name+'.ply'
    if os.path.exists(path):
        node.path=path
        oldCounter+=1
        pass
    else:
        # node.mesh=tl.ifc_to_mesh(node.ifcElement)
        name=node.name
        node.path=path
        try:
            o3d.io.write_triangle_mesh(node.path, node.mesh)
            newCounter+=1
        except:
            print("Export failed. Perhaps geometry creation failed or path is faulty?")
print (str(oldCounter)+' of '+str(len(nodelist))+ ' Nodes already have data!')
print (str(newCounter)+' of '+str(len(nodelist))+ ' Nodes newly created!')

64 of 64 Nodes already have data!
0 of 64 Nodes newly created!


## 4. CREATE RDF GRAPHS FOR EACH NODE AND EXPORT THEM TO .TTL
An RDF graph and .ttl file is created for all nodes in the session
(data itself is not stored in the graph, only metadata)

In [33]:
graph=tl.nodes_to_graph(nodelist)
graph.serialize(destination=sessionPath+"\\bimGraph.ttl", format='ttl')
print (str(len(nodelist))+' Nodes succesfully serialized in: '+sessionPath+"\\bimGraph.ttl")

64 Nodes succesfully serialized in: K:\Projects\2021-03 Project FWO SB Heinder\7.Data\2021-09 Testcase code\IFC\BIM-UF\bimGraph.ttl
